In [23]:
from src.data import math_tags
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import logging

In [31]:
_LOGGER = logging.getLogger()
_LOGGER.setLevel(logging.DEBUG)

In [11]:
tag_sequences = math_tags.get_sequences()

In [12]:
#getting 1 hot representation of sequences
#for each sequence with length N, you will have a corresponding tensor with shaep N by N_ELEMENTS representing set of sequences
one_hot_sequences = []
for sequence in tag_sequences:
    one_hot_sequences.append(torch.cat([torch.zeros(1, math_tags.N_ELEMENTS, dtype=torch.float).\
     scatter(1, torch.LongTensor(elm_set).view(1,-1), 1) for elm_set in sequence]))

In [84]:
n_epoch = 50
n_seq = 100

split = int(n_seq*0.8)
train_sequences = tag_sequences[:split]
train_targets =  one_hot_sequences[:split]

test_sequences = tag_sequences[split:n_seq]
test_targets = one_hot_sequences[split:n_seq]

loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=.9)

In [90]:
from src.model.set_sequence import SetSequenceModel

tag_embedding = torch.load('data/processed/math_tags/coocurance_count_pca_5.pt')
HIDDEN_DIM = 100
EMBEDDING_DIM = 2
model = SetSequenceModel(hidden_dim=HIDDEN_DIM,
                         n_class=math_tags.N_ELEMENTS,
                         embedding_dim=EMBEDDING_DIM)

In [92]:
test_losses = []
for sequence, target in zip(test_sequences, test_targets):
    model.hidden = model.init_hidden()
    logits = model(sequence)
    loss = loss_fn(logits[1:].view(-1),target[1:].view(-1))
    test_losses.append(loss.data)
_LOGGER.info("Validation Loss: {}".format(np.mean(test_losses)))

RuntimeError: Expected hidden[0] size (2, 10, 100), got (2, 1, 100)

In [63]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=.9)
losses = []

for epoch in range(n_epoch):
    curr_losses = []
    for sequence, target in zip(train_sequences,
                                train_targets):
        model.zero_grad()
        model.hidden = model.init_hidden()
        
        logits = model(sequence)
        loss = loss_fn(logits[1:].view(-1),target[1:].view(-1))
        curr_losses.append(loss.data)
        loss.backward()
        optimizer.step()
    mean_loss = np.mean(curr_losses)
    losses.append(mean_loss)
    _LOGGER.debug("epoch {}: {}".format(epoch, mean_loss))

DEBUG:root:epoch 0: 0.6880725622177124
DEBUG:root:epoch 1: 0.668692946434021
DEBUG:root:epoch 2: 0.6215929985046387
DEBUG:root:epoch 3: 0.4409300684928894
DEBUG:root:epoch 4: 0.2212485820055008
DEBUG:root:epoch 5: 0.13037517666816711
DEBUG:root:epoch 6: 0.0908234715461731
DEBUG:root:epoch 7: 0.06973027437925339
DEBUG:root:epoch 8: 0.05677608773112297
DEBUG:root:epoch 9: 0.048056792467832565
DEBUG:root:epoch 10: 0.04181313142180443
DEBUG:root:epoch 11: 0.03713825345039368
DEBUG:root:epoch 12: 0.03351728245615959
DEBUG:root:epoch 13: 0.030636567622423172
DEBUG:root:epoch 14: 0.02829449437558651
DEBUG:root:epoch 15: 0.026355843991041183
DEBUG:root:epoch 16: 0.024726685136556625
DEBUG:root:epoch 17: 0.023339856415987015
DEBUG:root:epoch 18: 0.022146105766296387
DEBUG:root:epoch 19: 0.021108541637659073
DEBUG:root:epoch 20: 0.020198997110128403
DEBUG:root:epoch 21: 0.019395630806684494
DEBUG:root:epoch 22: 0.01868123933672905
DEBUG:root:epoch 23: 0.01804211363196373
DEBUG:root:epoch 24: 0.0

In [64]:
test_losses = []
for sequence, target in zip(test_sequences, test_targets):
    model.hidden = model.init_hidden()
    logits = model(sequence)
    loss = loss_fn(logits[1:].view(-1),target[1:].view(-1))
    test_losses.append(loss.data)
_LOGGER.info("Validation Loss: {}".format(np.mean(test_losses)))

INFO:root:Validation Loss: 0.012033947743475437


In [68]:
i = 10
logits = model(test_sequences[i])
prediction = torch.sigmoid(model(test_sequences[i]))

In [69]:
print(test_sequences[i])

[[132, 336, 52], [52, 40], [52], [52], [132, 362, 52, 0], [336], [132, 52], [132, 52, 95], [132, 52, 216], [132, 52, 216, 95], [336, 52, 216]]


In [70]:
np.argsort(prediction.data.numpy(),axis=1)[:,:1650:-1]

array([[ 52,   4, 136,  15,  81, 112,  16, 216,  38,  37,  71, 171,  55],
       [ 52,   4, 136,  15,  38,  16, 112,  81, 216,  55,  71,  85,  21],
       [  4,  52, 136,  15,  38,  16, 112,  81,  55, 216,  85,  71, 132],
       [  4,  52, 136,  15,  38,  16, 112,  81,  55,  85, 216,  71, 132],
       [  4,  52, 136,  15,  38,  16, 112,  81,  55,  85, 216,  71, 132],
       [  4,  52, 136,  15,  38,  16, 112,  81,  55,  85, 216,  71, 132],
       [  4,  52, 136,  15,  38,  16, 112,  81,  55,  85, 216,  71, 132],
       [  4,  52, 136,  15,  38,  16, 112,  81,  55,  85, 216,  71, 132],
       [  4,  52, 136,  15,  38,  16, 112,  81,  55,  85, 216,  71, 132],
       [  4,  52, 136,  15,  38,  16, 112,  81,  55,  85, 216,  71, 132],
       [  4,  52, 136,  15,  38,  16, 112,  81,  55,  85, 216,  71, 132]])

In [52]:
np.max(prediction.data.numpy())

0.34843218